In [1]:
from pyblock2.driver.core import DMRGDriver, SymmetryTypes
from pyblock2 import tools
from block2 import SU2 as block2_SU2
from block2 import SZ as block2_SZ
from pathlib import Path
import CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import CAS.dmrghandler.src.dmrghandler as dmrghandler
from CAS.dmrghandler.src.dmrghandler.dmrg_looping import dmrg_central_loop
from DMRG_simulation.dmrg_simulation import get_ground_state_fci
import numpy as np
import CAS_Cropping.saveload_utils as sl
import CAS_Cropping.ferm_utils as feru
import CAS_Cropping.var_utils as varu
import pyscf
import scipy
import openfermion as of

In [2]:
L = 4
N = 4
TWOSZ = 0

driver = DMRGDriver(scratch="./tmp", symm_type=SymmetryTypes.SZ, n_threads=4)
driver.initialize_system(n_sites=L, n_elec=N, spin=TWOSZ)

In [3]:
t = 1
U = 1
mu = 0
s = 1
b = driver.expr_builder()

# hopping term
for j in range(1, 1 + s):
    b.add_term("cd", np.array([[[i, i + j], [i + j, i]] for i in range(L - j)]).flatten(), -t)
    b.add_term("CD", np.array([[[i, i + j], [i + j, i]] for i in range(L - j)]).flatten(), -t)

# onsite term
b.add_term("cdCD", np.array([[i, ] * 4 for i in range(L)]).flatten(), U)
# potential term
for i in range(L):
    for j in range(L):
        b.add_term("cdCD", np.array([[i], [i], [j], [j]]).flatten(), -mu)
print(b.data)
mpo = driver.get_mpo(b.finalize(), iprint=2)

 SU2 = 0 CONST E = 0.0000000000000000
 TERM cd ::
      0      1 = -1.0000000000000000
      1      0 = -1.0000000000000000
      1      2 = -1.0000000000000000
      2      1 = -1.0000000000000000
      2      3 = -1.0000000000000000
      3      2 = -1.0000000000000000
 TERM CD ::
      0      1 = -1.0000000000000000
      1      0 = -1.0000000000000000
      1      2 = -1.0000000000000000
      2      1 = -1.0000000000000000
      2      3 = -1.0000000000000000
      3      2 = -1.0000000000000000
 TERM cdCD ::
      0      0      0      0 = 1.0000000000000000
      1      1      1      1 = 1.0000000000000000
      2      2      2      2 = 1.0000000000000000
      3      3      3      3 = 1.0000000000000000
 TERM cdCD ::
      0      0      0      0 = 0.0000000000000000
 TERM cdCD ::
      0      0      1      1 = 0.0000000000000000
 TERM cdCD ::
      0      0      2      2 = 0.0000000000000000
 TERM cdCD ::
      0      0      3      3 = 0.0000000000000000
 TERM cdCD ::
      1   

In [4]:
def run_dmrg(driver, mpo):
    ket = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)
    bond_dims = [5] * 2 + [10] * 2 + [15] * 2 + [500] * 2
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
        thrds=thrds, cutoff=0, iprint=2)

energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)

Environment initialization | Nsites =     4 | Center =     0
 INIT-R <-- Site =    1 ..  Bmem =   160 B Rmem =   160 B T = 0.00
 INIT-R <-- Site =    0 ..  Bmem = 1.59 KB Rmem = 1.59 KB T = 0.00
Time init sweep =        0.005 | MaxBmem = 1.59 KB | MaxRmem = 1.59 KB
 | Tread = 0.000 | Twrite = 0.001 | Tfpread = 0.000 | Tfpwrite = 0.000 | data = 1.75 KB | cpsd = 1.34 KB | Tasync = 0.000


Sweep =    0 | Direction =  forward | Bond dimension =    5 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
 --> Site =    0-   1 .. Mmps =    4 Ndav =  18 E =     -3.5753656204 Error = 0.00e+00 FLOPS = 4.40e+06 Tdav = 0.01 T = 0.01
 --> Site =    1-   2 .. Mmps =    5 Ndav =   1 E =     -3.5753656204 Error = 1.84e-02 FLOPS = 1.59e+06 Tdav = 0.00 T = 0.00
 --> Site =    2-   3 .. Mmps =    5 Ndav =   4 E =     -3.5151664732 Error = 4.51e-06 FLOPS = 3.02e+05 Tdav = 0.00 T = 0.00
Time elapsed =      0.024 | E =      -3.5753656204 | DW = 1.84e-02
Time sweep =        0.024 | 26.9 KFLOP/SWP
 | Dmem = 2.16 KB

In [58]:
def run_dmrg(driver, mpo):
    ket = driver.get_random_mps(tag="KET", bond_dim=250, nroots=1)
    bond_dims = [250] * 4 + [500] * 4
    noises = [1e-4] * 4 + [1e-5] * 4 + [0]
    thrds = [1e-10] * 8
    return driver.dmrg(mpo, ket, n_sweeps=20, bond_dims=bond_dims, noises=noises,
        thrds=thrds, cutoff=0, iprint=2)

energies = run_dmrg(driver, mpo)
print('DMRG energy = %20.15f' % energies)

Environment initialization | Nsites =     4 | Center =     0
 INIT-R <-- Site =    1 ..  Bmem =   160 B Rmem =   160 B T = 0.00
 INIT-R <-- Site =    0 ..  Bmem = 1.59 KB Rmem = 1.59 KB T = 0.00
Time init sweep =        0.007 | MaxBmem = 1.59 KB | MaxRmem = 1.59 KB
 | Tread = 0.000 | Twrite = 0.001 | Tfpread = 0.000 | Tfpwrite = 0.000 | data = 1.75 KB | cpsd = 1.33 KB | Tasync = 0.000


Sweep =    0 | Direction =  forward | Bond dimension =  250 | Noise =  1.00e-04 | Dav threshold =  1.00e-10
 --> Site =    0-   1 .. Mmps =    4 Ndav =  14 E =     -2.9873384212 Error = 0.00e+00 FLOPS = 9.94e+05 Tdav = 0.01 T = 0.01
 --> Site =    1-   2 .. Mmps =    9 Ndav =   1 E =     -2.9873384212 Error = 0.00e+00 FLOPS = 9.50e+05 Tdav = 0.00 T = 0.00
 --> Site =    2-   3 .. Mmps =   12 Ndav =   1 E =     -2.9873384212 Error = 0.00e+00 FLOPS = 8.57e+05 Tdav = 0.00 T = 0.00
Time elapsed =      0.017 | E =      -2.9873384212 | DW = 0.00e+00
Time sweep =        0.017 | 5.91 KFLOP/SWP
 | Dmem = 1.84 KB